In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter

In [6]:
data = pd.read_excel('GeneExpressionCancer_training.xlsx')
val_data = pd.read_excel('GeneExpressionCancer_validation.xlsx')

In [4]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.preprocessing

In [ ]:
%load_ext tensorboard

In [12]:
import torch
from torch.utils.tensorboard import SummaryWriter

class MyNetwork(torch.nn.Module):

    def __init__(self, nFeatures):
        super().__init__()

        self.nFeatures = nFeatures
        self.batchSize = 500
        self.learningRate = 0.01
        self.epochs = 250

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(self.nFeatures, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 1),
            torch.nn.Sigmoid(),
        )

    def forward(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        return self.fc(x)
    
    def trainModel(self, trainData, trainLabels, validationData, validationLabels, logPath):
        # Configuring batches
        nTrainingSamples = trainData.shape[0]
        nTrainingBatches = nTrainingSamples // self.batchSize
        if nTrainingBatches * self.batchSize < nTrainingSamples:
            nTrainingBatches += 1

        nValidationSamples = validationData.shape[0]
        nValidationBatches = nValidationSamples // self.batchSize
        if nValidationBatches * self.batchSize < nValidationSamples:
            nValidationBatches += 1

        # Configuring device
        device = torch.device('cuda')
        self.to(device=device)

        # Configuring optimizer
        optimizer = torch.optim.SGD(self.parameters(), lr=self.learningRate)
        loss = torch.nn.BCELoss()

        # Creating logger
        writer = SummaryWriter(logPath)

        # Add model graph to TensorBoard
        sample_input_tensor = torch.randn(1, self.nFeatures, device=device)
        writer.add_graph(self, sample_input_tensor)

        # Training
        for epoch in range(self.epochs):
            self.train(mode=True)
            epochLoss = 0.0
            epochAccuracy = 0.0

            for batch in range(nTrainingBatches):
                optimizer.zero_grad()

                x = torch.tensor(trainData[batch*self.batchSize:(batch+1)*self.batchSize, :], 
                    device=device,
                    dtype=torch.float32
                )

                y = torch.tensor(trainLabels[batch*self.batchSize:(batch+1)*self.batchSize].reshape((-1,1)), 
                    device=device,
                    dtype=torch.float32
                )

                # Forward pass
                y_pred = self.fc(x)
                
                # Compute loss
                batchLoss = loss(y_pred, y)

                # Backpropagation
                batchLoss.backward()

                # Update parameters
                optimizer.step()

                # Accumulate loss
                epochLoss += batchLoss.item() * x.shape[0] / nTrainingSamples

                # Compute accuracy
                labels_pred = torch.round(y_pred)
                correct = (y == labels_pred).float()
                accuracy = correct.sum() / correct.numel()
                epochAccuracy += 100 * accuracy.item() * x.shape[0] / nTrainingSamples

            print(f'Epoch {epoch+1}/{self.epochs} - Loss: {epochLoss:.4f} - Accuracy: {epochAccuracy:.2f}%')

            writer.add_scalar('Loss/train', epochLoss, epoch)
            writer.add_scalar('Accuracy/train', epochAccuracy, epoch)

        writer.close()

    def save(self, path):
        torch.save(self, path)



In [5]:
trainingData = pd.read_excel('GeneExpressionCancer_training.xlsx').to_numpy()
trainingLabels = trainingData[:, -1]
trainingData = trainingData[:, :-1]

validationData = pd.read_excel('GeneExpressionCancer_validation.xlsx').to_numpy()
validationLabels = validationData[:, -1]
validationData = validationData[:, :-1]

testData = pd.read_excel('GeneExpressionCancer_test.xlsx').to_numpy()
testLabels = testData[:, -1]
testData = testData[:, :-1]



In [13]:
# Normalize data
scaler = sklearn.preprocessing.StandardScaler()
trainingData = scaler.fit_transform(trainingData)
validationData = scaler.transform(validationData)
testData = scaler.transform(testData)

model = MyNetwork(trainingData.shape[1])
model.trainModel(trainingData, trainingLabels, validationData, validationLabels, 'log/MyNetwork')
model.save('FinalModel')

C:\Users\Juan\AppData\Local\Temp\ipykernel_24200\640212053.py:22: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\Juan\AppData\Local\Temp\ipykernel_24200\640212053.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)


Epoch 1/250 - Loss: 0.6956 - Accuracy: 50.75%
Epoch 2/250 - Loss: 0.6948 - Accuracy: 51.31%
Epoch 3/250 - Loss: 0.6940 - Accuracy: 51.71%
Epoch 4/250 - Loss: 0.6932 - Accuracy: 52.20%
Epoch 5/250 - Loss: 0.6924 - Accuracy: 52.38%
Epoch 6/250 - Loss: 0.6917 - Accuracy: 52.64%
Epoch 7/250 - Loss: 0.6909 - Accuracy: 52.91%
Epoch 8/250 - Loss: 0.6901 - Accuracy: 53.28%
Epoch 9/250 - Loss: 0.6893 - Accuracy: 53.68%
Epoch 10/250 - Loss: 0.6885 - Accuracy: 53.88%
Epoch 11/250 - Loss: 0.6877 - Accuracy: 54.30%
Epoch 12/250 - Loss: 0.6868 - Accuracy: 54.40%
Epoch 13/250 - Loss: 0.6859 - Accuracy: 54.79%
Epoch 14/250 - Loss: 0.6850 - Accuracy: 55.30%
Epoch 15/250 - Loss: 0.6840 - Accuracy: 55.73%
Epoch 16/250 - Loss: 0.6830 - Accuracy: 56.23%
Epoch 17/250 - Loss: 0.6819 - Accuracy: 56.61%
Epoch 18/250 - Loss: 0.6807 - Accuracy: 57.18%
Epoch 19/250 - Loss: 0.6795 - Accuracy: 57.55%
Epoch 20/250 - Loss: 0.6781 - Accuracy: 57.90%
Epoch 21/250 - Loss: 0.6766 - Accuracy: 58.53%
Epoch 22/250 - Loss: 0

TypeError: Module._apply() missing 1 required positional argument: 'fn'

In [16]:
dir(MyNetwork)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_call_impl',
 '_get_backward_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_maybe_warn_non_full_backward_hook',
 '_named_members',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_version',
 'add_module',
 'apply',
 'bfloat16',
 'buffers',
 'children',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'get_buffer',
 'get_extra_state',
 'get_parameter',
 'get_submodule'